In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"

In [2]:
import abc
from collections import defaultdict
import json
from IPython.display import clear_output
from itertools import chain
import matplotlib.pyplot as plt
import numpy as np
import numpy.typing as npt
from pathlib import Path
import pickle
import tensorflow as tf
import tf_utilities as tfu
from tqdm.auto import tqdm
from typing import Iterable, Generator, Optional
import time
import wandb

from dnadb.datasets import Greengenes, Silva
from dnadb import dna, fasta, sample, taxonomy

from deepdna.data.dataset import Dataset
from deepdna.data.tokenizers import AbstractTaxonomyTokenizer, TopDownTaxonomyTokenizer
from deepdna.nn.models import custom_model, dnabert, load_model, taxonomy as tax_models
from deepdna.nn.utils import encapsulate_model
from deepdna.nn import layers, functional, utils

In [3]:
tfu.devices.select_gpu(0)

([PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')],
 [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')])

## Dataset

In [4]:
dataset = Dataset("/home/dwl2x/work/Datasets/Silva2/0")
train_fastas = tuple(map(sample.load_fasta, dataset.fasta_dbs(Dataset.Split.Train)))
train_tax = tuple(map(taxonomy.TaxonomyDb, dataset.taxonomy_dbs(Dataset.Split.Train)))
test_fastas = tuple(map(sample.load_fasta, dataset.fasta_dbs(Dataset.Split.Test)))
test_tax = tuple(map(taxonomy.TaxonomyDb, dataset.taxonomy_dbs(Dataset.Split.Test)))

In [5]:
for label in test_tax[0]:
    assert train_tax[0].contains_label(label)

In [6]:
tokenizer = TopDownTaxonomyTokenizer(depth=6)
for db in train_tax:
    tokenizer.add_labels(db)
tokenizer.build()

In [30]:
from deepdna.data.samplers import SampleSampler, SequenceSampler
from deepdna.nn.data_generators import _encode_sequences, BatchGenerator
from typing import Any, cast

class SequenceTaxonomyGenerator(BatchGenerator):
    def __init__(
        self,
        fasta_taxonomy_pairs: Iterable[tuple[sample.FastaSample, taxonomy.TaxonomyDb]],
        sequence_length: int,
        taxonomy_tokenizer: AbstractTaxonomyTokenizer,
        kmer: int = 1,
        subsample_size: int|None = None,
        batch_size: int = 32,
        batches_per_epoch: int = 100,
        augment_slide: bool = True,
        augment_ambiguous_bases: bool = True,
        balance: bool = False,
        shuffle: bool = True,
        rng: np.random.Generator = np.random.default_rng()
    ):
        super().__init__(
            batch_size=batch_size,
            batches_per_epoch=batches_per_epoch,
            shuffle=shuffle,
            rng=rng
        )
        fasta_samples, taxonomy_dbs = zip(*fasta_taxonomy_pairs)
        self.sample_sampler = SampleSampler(cast(tuple[sample.FastaSample, ...], fasta_samples))
        self.sequence_sampler = SequenceSampler(sequence_length, augment_slide)
        self.taxonomy_dbs: tuple[taxonomy.TaxonomyDb, ...] = cast(Any, taxonomy_dbs)
        self.kmer = kmer
        self.taxonomy_tokenizer = taxonomy_tokenizer
        self.subsample_size = subsample_size
        self.augment_ambiguous_bases = augment_ambiguous_bases
        self.balance = balance

    @property
    def sequence_length(self) -> int:
        return self.sequence_sampler.sequence_length

    def generate_batch(
        self,
        rng: np.random.Generator
    ) -> tuple[npt.NDArray[np.int32], npt.NDArray[np.int32]]:
        subsample_size = self.subsample_size or 1
        sequences = np.empty((self.batch_size, subsample_size), dtype=f"<U{self.sequence_length}")
        sample_ids = np.empty(self.batch_size, dtype=np.int32)
        sequence_ids = [None] * self.batch_size
        label_ids = np.empty((self.batch_size, subsample_size, self.taxonomy_tokenizer.depth), dtype=np.int32)
        samples = self.sample_sampler.sample_with_ids(self.batch_size, self.balance, rng)
        for i, (sample_id, sample) in enumerate(samples):
            tax_db = self.taxonomy_dbs[sample_id]
            sequence_info = tuple(self.sequence_sampler.sample_with_ids(sample, subsample_size, rng))
            sequence_ids[i], sequences[i] = zip(*sequence_info)
            sample_ids[i] = sample_id
            label_ids[i] = [self.taxonomy_tokenizer.tokenize_label(tax_db.fasta_id_to_label(fasta_id)) for fasta_id in sequence_ids[i]]
        sequences = _encode_sequences(sequences, self.augment_ambiguous_bases, self.rng)
        if self.subsample_size is None:
            sequences = np.squeeze(sequences, axis=1)
            label_ids = np.squeeze(label_ids, axis=1)
        sequences = sequences.astype(np.int32)
        if self.kmer > 1:
            sequences = dna.encode_kmers(sequences, self.kmer, not self.augment_ambiguous_bases).astype(np.int32) # type: ignore
        return sample_ids, sequence_ids, sequences, tuple(label_ids.T)[-1]

    def reduce_batch(self, batch):
        # remove sample IDs and sequence IDs
        return batch[2:]

## Model

In [4]:
# api = wandb.Api()
run = wandb.init(project="dnabert-taxonomy", name="topdown (extended) 64d-150l", resume="must", id="k137vfxy")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sirdavidludwig. Use `wandb login --relogin` to force relogin


In [9]:
path = run.use_artifact("sirdavidludwig/dnabert-taxonomy/dnabert-taxonomy-topdown-64d-150l:v0").download()
taxonomy_model = load_model(path)

wandb:   4 of 4 files downloaded.  


In [13]:
pretrained_base = taxonomy_model.base.base

In [14]:
base = dnabert.DnaBertModel(
    sequence_length=pretrained_base.sequence_length,
    kmer=pretrained_base.kmer,
    embed_dim=pretrained_base.embed_dim,
    stack=2*pretrained_base.stack,
    num_heads=pretrained_base.num_heads,
)

In [15]:
base.stack

16

In [21]:
# Copy pretrained weights into the first stack of transformers of the new model
for a, b in zip(pretrained_base.model.layers[2:10], base.model.layers[2:10]):
    b.set_weights(a.get_weights())

In [26]:
encoder = dnabert.DnaBertEncoderModel(base, 256)
model = tax_models.TopDownTaxonomyClassificationModel(encoder, taxonomy_model.taxonomy_tokenizer)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4))

In [31]:
common_args = dict(
    sequence_length = 150,
    kmer = 3,
    taxonomy_tokenizer = model.taxonomy_tokenizer,
    subsample_size=None,
    batch_size = 256,
)

train_data = SequenceTaxonomyGenerator(
    zip(train_fastas, train_tax),
    batches_per_epoch=100,
    **common_args)
test_data = SequenceTaxonomyGenerator(
    zip(test_fastas, test_tax),
    batches_per_epoch=20,
    shuffle=False,
    **common_args)

In [29]:
wandb_callback = wandb.keras.WandbCallback(save_model=False)
wandb_callback.save_model_as_artifact = False
checkpoint = tf.keras.callbacks.ModelCheckpoint("logs/models/dnabert_taxonomy_topdown_extended", save_best=False)

In [35]:
model.fit(train_data, validation_data=test_data, epochs=4000, initial_epoch=3000, callbacks=[wandb_callback, checkpoint])

Epoch 3001/4000
100/100 [==============================] - ETA: 0s - loss: 0.9252 - sparse_categorical_accuracy: 0.7879

INFO:tensorflow:Assets written to: logs/models/dnabert_taxonomy_topdown_extended/assets


INFO:tensorflow:Assets written to: logs/models/dnabert_taxonomy_topdown_extended/assets
/opt/conda/lib/python3.10/site-packages/keras/engine/functional.py:1384: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/opt/conda/lib/python3.10/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


100/100 [==============================] - 81s 804ms/step - loss: 0.9252 - sparse_categorical_accuracy: 0.7879 - val_loss: 0.7913 - val_sparse_categorical_accuracy: 0.8160
Epoch 3002/4000
100/100 [==============================] - ETA: 0s - loss: 0.9466 - sparse_categorical_accuracy: 0.7850

INFO:tensorflow:Assets written to: logs/models/dnabert_taxonomy_topdown_extended/assets


...

100/100 [==============================] - 81s 814ms/step - loss: 0.8795 - sparse_categorical_accuracy: 0.7974 - val_loss: 0.7492 - val_sparse_categorical_accuracy: 0.8244
Epoch 3901/4000
100/100 [==============================] - ETA: 0s - loss: 0.9007 - sparse_categorical_accuracy: 0.7959

INFO:tensorflow:Assets written to: logs/models/dnabert_taxonomy_topdown_extended/assets


INFO:tensorflow:Assets written to: logs/models/dnabert_taxonomy_topdown_extended/assets
/opt/conda/lib/python3.10/site-packages/keras/engine/functional.py:1384: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/opt/conda/lib/python3.10/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


100/100 [==============================] - 84s 847ms/step - loss: 0.9007 - sparse_categorical_accuracy: 0.7959 - val_loss: 0.7462 - val_sparse_categorical_accuracy: 0.8248
Epoch 3902/4000
 53/100 [==============>...............] - ETA: 24s - loss: 0.8695 - sparse_categorical_accuracy: 0.8014

KeyboardInterrupt: 

In [6]:
model = load_model("logs/models/dnabert_taxonomy_topdown_extended/")

In [8]:
model.save(Path(wandb.run.dir) / "model")

INFO:tensorflow:Assets written to: /home/dwl2x/.cache/wandb/wandb/run-20230927_234804-k137vfxy/files/model/assets


INFO:tensorflow:Assets written to: /home/dwl2x/.cache/wandb/wandb/run-20230927_234804-k137vfxy/files/model/assets
/opt/conda/lib/python3.10/site-packages/keras/engine/functional.py:1384: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/opt/conda/lib/python3.10/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [9]:
a = wandb.Artifact(name="dnabert-taxonomy-topdown-deep-64d-150l", type="model")

In [10]:
a.add_dir(Path(wandb.run.dir) / "model")

wandb: Adding directory to artifact (/home/dwl2x/.cache/wandb/wandb/run-20230927_234804-k137vfxy/files/model)... Done. 0.1s


In [11]:
run.log_artifact(a)

<Artifact dnabert-taxonomy-topdown-deep-64d-150l>

In [12]:
run.finish()

best_epoch,3857
best_val_loss,0.73911
epoch,3900
loss,0.90071
sparse_categorical_accuracy,0.79594
val_loss,0.74622
val_sparse_categorical_accuracy,0.8248
